In [1]:
secret_key = " sk-proj-pqEfsvNNHPlhNAvPbJu3T3BlbkFJqfdIWBNRHYBPdTSNDYLZ"
secret_key

' sk-proj-pqEfsvNNHPlhNAvPbJu3T3BlbkFJqfdIWBNRHYBPdTSNDYLZ'

In [2]:
import os
import openai
os.environ['OPENAI_API_KEY'] = secret_key
openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
# !pip install langchain

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import UnstructuredExcelLoader
from langchain.llms import OpenAI
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

c:\Users\ahmed\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [5]:
client = openai.OpenAI(api_key=secret_key)

In [6]:
pdf_directory = 'project_files/questions_folder/'
questions = []

In [7]:
for filename in os.listdir(pdf_directory):
    if filename.endswith('.pdf'):
        file_path = os.path.join(pdf_directory, filename)
        question = PyPDFLoader(file_path).load()
        questions.append(question)
questions = questions[:len(questions)//2]

In [8]:
rubric = PyPDFLoader('project_files/rubric.pdf').load()

In [9]:
chunk_size = 1000
chunk_overlap = 50

In [10]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""]
)

In [11]:
persist_directory = 'content/vectorstore/chroma/'

In [12]:
splits = [r_splitter.split_documents(q) for q in questions]
# logging.debug(f"Total document splits: {len(splits)}")
questions = splits[0]
for i in splits:
    if i != splits[0]:
        questions += i
questions += r_splitter.split_documents(rubric)

In [13]:

vectordb = Chroma.from_documents(
    documents = questions,
    embedding = embedding,
    persist_directory = persist_directory
)

In [14]:
llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)
retriever = VectorStoreRetriever(vectorstore=vectordb)
qachain = RetrievalQA.from_llm(llm=llm, retriever=retriever)

c:\Users\ahmed\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [15]:
import ipywidgets as widgets
from IPython.display import display

upload_widget = widgets.FileUpload(
    accept='.py',  # Accept only .py files
    multiple=False  # Do not allow multiple file uploads
)

# Display the widget
display(upload_widget)

def read_file(change):
    uploaded_file = list(upload_widget.value.values())[0]
    content = uploaded_file['content'].decode('utf-8')

    system_context = "You are a python teaching assistant. Your goal is to help students with their assignments and if they give you code you grade them based on the rubric pdf in the database and the questions of that assignment number in the database."

    student_code = content
    assignment_number = 2
    
    query = "what are the questions in assignment 2?"
    
    response = qachain({'query': query})
    print(response['result'])
    
    detailed_feedback_requested = True

    if detailed_feedback_requested:
        detailed_feedback_query = f"{system_context}\n\nStudent code:\n{student_code}\n\nWhat are the areas of improvement in assignment {assignment_number}?"
        detailed_feedback_response = qachain({'query': detailed_feedback_query})
        print(detailed_feedback_response['result'])

upload_widget.observe(read_file, names='value')


FileUpload(value={}, accept='.py', description='Upload')

c:\Users\ahmed\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 I don't know, as the context does not mention anything about assignment 2.

1. Lack of comments and user messages: The student code does not have any comments or user messages, which makes it difficult for others to understand the code and its purpose.

2. Use of hard-coded values: In problem 1 and 2, the student has used a specific list of numbers instead of allowing the user to input their own list. This limits the flexibility and usability of the code.

3. Inefficient use of data types: In problem 2, the student has used a for loop and an if statement to create a new list without using the set data type, which was specifically mentioned in the assignment instructions.

4. Lack of error handling: In problem 3 and 5, the student has not included any error handling for incorrect user inputs, which could lead to unexpected results or errors.

5. Incomplete solutions: In problem 4, the student has not fully completed the task as the code only calculates the approximate log base 2 of a n